# 1. SETTING
- 환경 설정 및 데이터 URL 불러오기

In [ ]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
key = '' # 개인정보이므로 삭제했습니다.
myURI = 'http://openapi.seoul.go.kr:8088/' + key + '/xml/TbVwShopping/1/1000/'

In [ ]:
print(myURI)

# 2. GET OPENAPI

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen

def get_openapi(url):
  global df # 전역 변수 설정

  html = urlopen(url)
  bs = BeautifulSoup(html.read(), 'lxml-xml') # html.parser보다 더 깨끗한 결과 제공, 한글 스트링 호환

  itemlist = bs.find_all("row")

  df = pd.DataFrame() # 데이터를 저장할 빈 dataframe 생성

  for t in itemlist:
    try:
      language = t.find('LANG_CODE_ID').get_text()
      name = t.find('POST_SJ').get_text()
      address = t.find('ADDRESS').get_text()
      new_address = t.find('NEW_ADDRESS').get_text()
      tel = t.find('CMMN_TELNO').get_text()
      web_url = t.find('CMMN_HMPG_URL').get_text()
      use_time = t.find('CMMN_USE_TIME').get_text()
      traffic_info = t.find('SUBWAY_INFO').get_text()
    
    except: # 제공되지 않는(비어있는) 항목일 경우, 오류없이 ""로 받아오기 
      language = ""
      name = ""
      address = ""
      new_address = ""
      tel = ""
      web_url = ""
      use_time = ""
      traffic_info = ""
    
    temp = pd.DataFrame(([[language, name, address, new_address, tel, web_url, use_time, traffic_info]]), 
                        columns = ["언어", "상호명", "주소", "신주소", "전화번호", "웹사이트", 
                                    "운영시간", "교통정보"])
    df = pd.concat([df,temp])

  df = df.reset_index(drop=True)
  return df

In [ ]:
get_openapi(myURI)

,언어,상호명,주소,신주소,전화번호,웹사이트,운영시간,교통정보
0,zh-TW,東大門購物區,100-951 首爾市中區乙支路6街,04564 首爾市中區獎忠壇路253,02-120,http://www.ddm-mall.co.kr/,營業時間依店家而有不同,地鐵1、4號線東大門站8號出口出來，步行5分鐘\r\n地鐵2、4、5號線東大門歷史文化公園站...
1,zh-CN,南大门市场,100-804 首尔中区南仓洞 49-1,04529 首尔市中区南大门市场4街21 (南仓洞),+82-2-753-2805,http://namdaemunmarket.co.kr/,00:00 - 23:00\r\n各卖场营业时间不同,"4号线 会贤站5,6,7号出口步行5分钟"
2,zh-CN,东大门综合市场,,,+82-2-2262-0114,http://www.ddm-mall.com/,因卖场而异,"1,4号线 东大门站4号出口"
3,zh-CN,东大门昌信洞玩具批发市场,110-841 서울 종로구 창신동 390-29,03120 서울 종로구 종로52길 36 (창신동),+82-2-120,,各卖场营业时间不同,"1号线 东大门站从4号出口出来,第二个右侧巷子\r\n2号线 东大门历史文化公园站2号出口,..."
4,ja,COEX MALL(コエックスモール),135-975 ソウル市江南区三成洞 159,06164 ソウル市カンナム(江南)区ヨンドンデロ513 (三成洞),+82-2-6002-5300,https://www.starfield.co.kr/coexmall/main.do,店舗により異なる(ホームページ参照),地下鉄2号線「サムソン(三成)」駅5番、6番出口
...,...,...,...,...,...,...,...,...
620,ko,쌈지길,서울 종로구 관훈동 38,"03146 서울 종로구 인사동길 44 (관훈동, 쌈지길)",02-736-0088,http://m.smartstore.naver.com/ssamzigil,10:30 ~ 20:30,3호선 안국역 6번 출구에서 245m
621,zh-CN,解放村新兴市场,首尔龙山区龙山洞2街1-480,04337 首尔龙山区新兴路95-9（龙山洞2街） 解放村新兴市场,+82-2-754-7389,http://www.letshbc.com,各个商店营业时间不同,绿莎坪站2号出口
622,en,Seoul Central Market,"442, Hwanghak-dong, Jung-gu, Seoul","04576 36, Toegye-ro 85-gil, Jung-gu, Seoul",+82-2-2232-9559,,Daily 07:00 - 21:00,
623,ko,아모레 플래그십 스토어 (성수),서울 성동구 성수동2가 277-52,"04797 서울 성동구 아차산로11길 7 (성수동2가, (주)아세아이엔티)",02-469-8600,https://www.amore-seongsu.com/main,10:30 ~ 20:30,성수역 2번 출구에서 264m


# 3. PREPROCESSING
  1. 언어 칼럼을 통해 한국어/영어/일본어/중국어 데이터 가져오기
  2. 언어 칼럼의 데이터를 한글로 대치 (ex. ko -> 한국어)
  3. 언어별 구 추출

In [ ]:
seoul_shopping_df = df

In [ ]:
# 언어 칼럼을 통해 한국어/영어/일본어/중국어 데이터만 가져오기
language = ['ko', 'en', 'ja', 'zh-CN'] # 한국어, 영어, 일본어, 중국어
seoul_shopping_df = seoul_shopping_df[seoul_shopping_df['언어'].isin(language)] # 4개 국어 데이터만 가져오기
seoul_shopping_df = seoul_shopping_df.reset_index(drop=True)

In [ ]:
# 언어에 따라 데이터 대치 (ex. ko -> 한국어)
for i in range(len(seoul_shopping_df)):
  if seoul_shopping_df['언어'][i] == 'ko': # 한국어
    seoul_shopping_df['언어'][i] = '한국어'
  elif seoul_shopping_df['언어'][i] == 'en': # 영어
    seoul_shopping_df['언어'][i] = '영어'
  elif seoul_shopping_df['언어'][i] == 'ja': # 일본어
    seoul_shopping_df['언어'][i] = '일본어'
  else: # 중국어
    seoul_shopping_df['언어'][i] = '중국어'

seoul_shopping_df.head(5)

,언어,상호명,주소,신주소,전화번호,웹사이트,운영시간,교통정보
0,중국어,南大门市场,100-804 首尔中区南仓洞 49-1,04529 首尔市中区南大门市场4街21 (南仓洞),+82-2-753-2805,http://namdaemunmarket.co.kr/,00:00 - 23:00\r\n各卖场营业时间不同,"4号线 会贤站5,6,7号出口步行5分钟"
1,중국어,东大门综合市场,,,+82-2-2262-0114,http://www.ddm-mall.com/,因卖场而异,"1,4号线 东大门站4号出口"
2,중국어,东大门昌信洞玩具批发市场,110-841 서울 종로구 창신동 390-29,03120 서울 종로구 종로52길 36 (창신동),+82-2-120,,各卖场营业时间不同,"1号线 东大门站从4号出口出来,第二个右侧巷子\r\n2号线 东大门历史文化公园站2号出口,..."
3,일본어,COEX MALL(コエックスモール),135-975 ソウル市江南区三成洞 159,06164 ソウル市カンナム(江南)区ヨンドンデロ513 (三成洞),+82-2-6002-5300,https://www.starfield.co.kr/coexmall/main.do,店舗により異なる(ホームページ参照),地下鉄2号線「サムソン(三成)」駅5番、6番出口
4,한국어,금천 관광 추천코스,서울특별시 금천구 가산동,서울특별시 금천구 일대,02-807-4800,,,1호선(경부선) 가산디지털단지역 \r\n1호선(경부선) 독산역 \r\n7호선...


In [ ]:
# 주소에서 구 추출
seoul_shopping_df['구'] = ''


for i in range(len(seoul_shopping_df)):
  # '주소' 칼럼을 기준으로 하되, 비어있으면 '신주소' 칼럼을 이용해서 전체 주소 불러오기
  full_address = seoul_shopping_df['주소'][i]
  if full_address == ' ':
    full_address = seoul_shopping_df['신주소'][i]


  if seoul_shopping_df['언어'][i] == '한국어': # 한국어 주소 추출
    try:
      seoul_shopping_df['구'][i] = full_address.split(' ')[2]
      if seoul_shopping_df['구'][i][-1] != '구':
        seoul_shopping_df['구'][i] = 'error'
    except:
      seoul_shopping_df['구'][i] = 'error'

  elif seoul_shopping_df['언어'][i] == '영어': # 영어 주소 추출
    try:
      address = seoul_shopping_df['주소'][i].split(',')
      seoul_shopping_df['구'][i] = address[['-gu' in address[i] for i in range(len(address))].index(True)].replace(' ', '')
    except:
      seoul_shopping_df['구'][i] = 'error'

  elif seoul_shopping_df['언어'][i] == '일본어': # 일본어 주소 추출
    try:
      address1 = full_address.split('市')
      address2 = address1[1].split('区')
      seoul_shopping_df['구'][i] = address2[0]+'区'
    except:
      seoul_shopping_df['구'][i] = 'error'

  else: # 중국어 주소 추출
    if '市' in full_address:
      address1 = full_address.split('市')
      if '区' in full_address:
        address2 = address1[1].split('区')
        seoul_shopping_df['구'][i] = address2[0] + '区'
      elif '區' in full_address:
        address2 = address1[1].split('區')
        seoul_shopping_df['구'][i] = address2[0] + '区'
      else:
        seoul_shopping_df['구'][i] = 'error'
      
    elif '首' in full_address:
      address1 = full_address.split('首')
      if '区' in full_address:
        address2 = address1[1].split('区')
        seoul_shopping_df['구'][i] = address2[0][1:] + '区'
      elif '區' in full_address:
        address2 = address1[1].split('區')
        seoul_shopping_df['구'][i] = address2[0][1:] + '区'
      else:

        seoul_shopping_df['구'][i] = 'error'

In [ ]:
seoul_shopping_df

,언어,상호명,주소,신주소,전화번호,웹사이트,운영시간,교통정보,구
0,중국어,南大门市场,100-804 首尔中区南仓洞 49-1,04529 首尔市中区南大门市场4街21 (南仓洞),+82-2-753-2805,http://namdaemunmarket.co.kr/,00:00 - 23:00\r\n各卖场营业时间不同,"4号线 会贤站5,6,7号出口步行5分钟",中区
1,중국어,东大门综合市场,,,+82-2-2262-0114,http://www.ddm-mall.com/,因卖场而异,"1,4号线 东大门站4号出口",
2,중국어,东大门昌信洞玩具批发市场,110-841 서울 종로구 창신동 390-29,03120 서울 종로구 종로52길 36 (창신동),+82-2-120,,各卖场营业时间不同,"1号线 东大门站从4号出口出来,第二个右侧巷子\r\n2号线 东大门历史文化公园站2号出口,...",
3,일본어,COEX MALL(コエックスモール),135-975 ソウル市江南区三成洞 159,06164 ソウル市カンナム(江南)区ヨンドンデロ513 (三成洞),+82-2-6002-5300,https://www.starfield.co.kr/coexmall/main.do,店舗により異なる(ホームページ参照),地下鉄2号線「サムソン(三成)」駅5番、6番出口,江南区
4,한국어,금천 관광 추천코스,서울특별시 금천구 가산동,서울특별시 금천구 일대,02-807-4800,,,1호선(경부선) 가산디지털단지역 \r\n1호선(경부선) 독산역 \r\n7호선...,금천구
...,...,...,...,...,...,...,...,...,...
588,한국어,쌈지길,서울 종로구 관훈동 38,"03146 서울 종로구 인사동길 44 (관훈동, 쌈지길)",02-736-0088,http://m.smartstore.naver.com/ssamzigil,10:30 ~ 20:30,3호선 안국역 6번 출구에서 245m,종로구
589,중국어,解放村新兴市场,首尔龙山区龙山洞2街1-480,04337 首尔龙山区新兴路95-9（龙山洞2街） 解放村新兴市场,+82-2-754-7389,http://www.letshbc.com,各个商店营业时间不同,绿莎坪站2号出口,龙山区
590,영어,Seoul Central Market,"442, Hwanghak-dong, Jung-gu, Seoul","04576 36, Toegye-ro 85-gil, Jung-gu, Seoul",+82-2-2232-9559,,Daily 07:00 - 21:00,,Jung-gu
591,한국어,아모레 플래그십 스토어 (성수),서울 성동구 성수동2가 277-52,"04797 서울 성동구 아차산로11길 7 (성수동2가, (주)아세아이엔티)",02-469-8600,https://www.amore-seongsu.com/main,10:30 ~ 20:30,성수역 2번 출구에서 264m,성동구


In [ ]:
seoul_shopping_df.to_excel('/content/drive/MyDrive/빅데이터통계분석/data/seoul_shopping.xlsx')